In [1]:
# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [2]:
# Veri Seti Yükleme
file_path = r"C:\Users\hsynt\Downloads/combined_wine_formatted_dataset.csv"
data = pd.read_csv(file_path)

In [3]:

# Kategorik sütunları dönüştürme
data_encoded = pd.get_dummies(data, columns=["type"], drop_first=True)

# Hedef Değişkenin Sayısal Değerlere Dönüştürülmesi
label_encoder = LabelEncoder()
data_encoded["quality_label"] = label_encoder.fit_transform(data_encoded["quality_label"])

# Özellik ve Hedef Değişkenlerin Ayrılması
X = data_encoded.drop(columns=["quality_label", "quality"])
y = data_encoded["quality_label"]

# Özelliklerin Ölçeklendirilmesi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Eğitim ve Test Setlerinin Oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# PCA ve LDA Uygulaması
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Modellerin Tanımlanması
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Support Vector Machine": SVC(probability=True, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

# FCNN Modelinin Tanımlanması
def create_fcnn_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Eğitim ve test sürelerini tutacak bir liste
time_results_combined = []

# Modellerin eğitim ve test sürelerini ölçme
for model_name, model in models.items():
    # PCA ve LDA'siz
    start_time = time.time()
    model.fit(X_train, y_train)  # Eğitim
    train_end_time = time.time()
    start_test_time = time.time()
    model.predict(X_test)  # Test
    test_end_time = time.time()

    time_results_combined.append({
        "Model": model_name,
        "Reduction": "No PCA/LDA",
        "Training Time(s)": train_end_time - start_time,
        "Test Time(s)": test_end_time - start_test_time
    })

    # PCA'lı
    start_time = time.time()
    model.fit(X_train_pca, y_train)
    train_end_time = time.time()
    start_test_time = time.time()
    model.predict(X_test_pca)
    test_end_time = time.time()

    time_results_combined.append({
        "Model": model_name,
        "Reduction": "With PCA",
        "Training Time(s)": train_end_time - start_time,
        "Test Time(s)": test_end_time - start_test_time
    })

    # LDA'lı
    start_time = time.time()
    model.fit(X_train_lda, y_train)
    train_end_time = time.time()
    start_test_time = time.time()
    model.predict(X_test_lda)
    test_end_time = time.time()

    time_results_combined.append({
        "Model": model_name,
        "Reduction": "With LDA",
        "Training Time(s)": train_end_time - start_time,
        "Test Time(s)": test_end_time - start_test_time
    })

    # K-Fold Cross Validation Zaman Ölçümü
    for reduction, X_data in zip(
        ["No PCA/LDA (K-Fold)", "With PCA (K-Fold)", "With LDA (K-Fold)"], 
        [X_train, X_train_pca, X_train_lda]):
        
        skf = StratifiedKFold(n_splits=5)
        total_train_time = 0
        total_test_time = 0

        for train_index, test_index in skf.split(X_data, y_train):
            X_fold_train, X_fold_test = X_data[train_index], X_data[test_index]
            y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]

            # Eğitim süresi
            start_time = time.time()
            model.fit(X_fold_train, y_fold_train)
            train_end_time = time.time()

            # Test süresi
            start_test_time = time.time()
            model.predict(X_fold_test)
            test_end_time = time.time()

            total_train_time += (train_end_time - start_time)
            total_test_time += (test_end_time - start_test_time)

        time_results_combined.append({
            "Model": model_name,
            "Reduction": reduction,
            "Training Time(s)": total_train_time,
            "Test Time(s)": total_test_time
        })

# FCNN Eğitim ve Test Süresi Ölçümü
for reduction, X_data, X_test_reduction in zip(
    ["No PCA/LDA", "With PCA", "With LDA"], 
    [X_train, X_train_pca, X_train_lda], 
    [X_test, X_test_pca, X_test_lda]):

    fcnn_model = create_fcnn_model(X_data.shape[1])
    
    start_time = time.time()
    fcnn_model.fit(X_data, y_train, epochs=10, batch_size=32, verbose=0)
    train_end_time = time.time()
    
    start_test_time = time.time()
    fcnn_model.predict(X_test_reduction)
    test_end_time = time.time()

    time_results_combined.append({
        "Model": "FCNN",
        "Reduction": reduction,
        "Training Time(s)": train_end_time - start_time,
        "Test Time(s)": test_end_time - start_test_time
    })

# FCNN K-Fold için Zaman Ölçümü
for reduction, X_data in zip(
    ["No PCA/LDA (K-Fold)", "With PCA (K-Fold)", "With LDA (K-Fold)"], 
    [X_train, X_train_pca, X_train_lda]):
    
    skf = StratifiedKFold(n_splits=5)
    total_train_time = 0
    total_test_time = 0

    for train_index, test_index in skf.split(X_data, y_train):
        X_fold_train, X_fold_test = X_data[train_index], X_data[test_index]
        y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]

        # Eğitim süresi
        start_time = time.time()
        fcnn_model = create_fcnn_model(X_fold_train.shape[1])
        fcnn_model.fit(X_fold_train, y_fold_train, epochs=10, batch_size=32, verbose=0)
        train_end_time = time.time()

        # Test süresi
        start_test_time = time.time()
        fcnn_model.predict(X_fold_test)
        test_end_time = time.time()

        total_train_time += (train_end_time - start_time)
        total_test_time += (test_end_time - start_test_time)

    time_results_combined.append({
        "Model": "FCNN",
        "Reduction": reduction,
        "Training Time(s)": total_train_time,
        "Test Time(s)": total_test_time
    })

# Sonuçları DataFrame'e dönüştürme
time_results_combined_df = pd.DataFrame(time_results_combined)

# Sonuçları görüntüleme
time_results_combined_df

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,Model,Reduction,Training Time(s),Test Time(s)
0,Random Forest,No PCA/LDA,0.694099,0.015314
1,Random Forest,With PCA,0.573412,0.017022
2,Random Forest,With LDA,0.543198,0.016342
3,Random Forest,No PCA/LDA (K-Fold),2.872473,0.058768
4,Random Forest,With PCA (K-Fold),2.232306,0.070337
5,Random Forest,With LDA (K-Fold),2.212982,0.068343
6,Logistic Regression,No PCA/LDA,0.036586,0.000000
7,Logistic Regression,With PCA,0.009511,0.000000
8,Logistic Regression,With LDA,0.007510,0.000000
9,Logistic Regression,No PCA/LDA (K-Fold),0.140379,0.002503
